In [ ]:
import tensorflow as tf
import numpy as np
import json
import os

In [2]:
RANDOM_SEED = 42

CWD = os.getcwd()

LABEL_PATH = os.path.join(CWD, "data", "labels.json")
TRAIN_DATASET_PATH = os.path.join(CWD, "data", "left_train.csv")
TEST_DATASET_PATH = os.path.join(CWD, "data", "left_test.csv")

MODEL_SAVE_PATH = os.path.join(CWD, "models", "gesture_classifier_left.keras")
TLITE_SAVE_PATH = os.path.join(CWD, "models", "gesture_classifier_left.tflite")

NUM_CLASSES = len(json.load(open(LABEL_PATH, "r")))

# Dataset reading

In [3]:
X_train_dataset = np.loadtxt(TRAIN_DATASET_PATH, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
X_test_dataset = np.loadtxt(TEST_DATASET_PATH, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [4]:
y_train_dataset = np.loadtxt(TRAIN_DATASET_PATH, delimiter=',', dtype='int32', usecols=(0))
y_test_dataset = np.loadtxt(TEST_DATASET_PATH, delimiter=',', dtype='int32', usecols=(0))

In [5]:
X_train = X_train_dataset
X_test = X_test_dataset
y_train = y_train_dataset
y_test = y_test_dataset

# Model building

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [7]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    MODEL_SAVE_PATH, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

# Convert model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(TLITE_SAVE_PATH, 'wb').write(tflite_quantized_model)